## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%%capture
!apt-get update

In [3]:
#https://stackoverflow.com/questions/50213021/best-practice-for-upgrading-cuda-and-cudnn-for-tensorflow
!sudo apt-get --purge remove cuda
!sudo apt-get autoremove
!dpkg --list |grep "^rc" | cut -d " " -f 3 | xargs sudo dpkg --purge

/bin/bash: sudo: command not found


/bin/bash: sudo: command not found


xargs: sudo: No such file or directory


In [4]:
#%%capture
!apt-get install -y cuda-command-line-tools-11-1 cuda-compiler-11-1 cuda-cudart-11-1 libcudnn8=8.0.5.39-1+cuda11.1 libcudnn8-dev=8.0.5.39-1+cuda11.1

The following additional packages will be installed:
  cuda-cudart-dev-11-1 cuda-cuobjdump-11-1 cuda-cupti-11-1 cuda-cupti-dev-11-1
  cuda-driver-dev-11-1 cuda-gdb-11-1 cuda-memcheck-11-1 cuda-nvcc-11-1
  cuda-nvdisasm-11-1 cuda-nvprof-11-1 cuda-nvprune-11-1 cuda-nvtx-11-1
  cuda-sanitizer-11-1


The following NEW packages will be installed:
  cuda-command-line-tools-11-1 cuda-compiler-11-1 cuda-cudart-11-1
  cuda-cudart-dev-11-1 cuda-cuobjdump-11-1 cuda-cupti-11-1 cuda-cupti-dev-11-1
  cuda-driver-dev-11-1 cuda-gdb-11-1 cuda-memcheck-11-1 cuda-nvcc-11-1
  cuda-nvdisasm-11-1 cuda-nvprof-11-1 cuda-nvprune-11-1 cuda-nvtx-11-1
  cuda-sanitizer-11-1
The following packages will be upgraded:
  libcudnn8 libcudnn8-dev
2 upgraded, 16 newly installed, 0 to remove and 33 not upgraded.
Need to get 984 MB of archives.
After this operation, 1155 MB of additional disk space will be used.
0% [Working]

Get:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  cuda-cudart-11-1 11.1.74-1 [133 kB]
Get:2 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  cuda-driver-dev-11-1 11.1.74-1 [25.4 kB]
Get:3 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  cuda-cudart-dev-11-1 11.1.74-1 [1734 kB]
2% [3 cuda-cudart-dev-11-1 27.5 kB/1734 kB 2%]

2% [3 cuda-cudart-dev-11-1 1734 kB/1734 kB 100%]

2% [3 cuda-cudart-dev-11-1 1734 kB/1734 kB 100%]

...

Setting up cuda-nvprof-11-1 (11.1.105-1) ...
Setting up cuda-nvcc-11-1 (11.1.105-1) ...


Setting up cuda-compiler-11-1 (11.1.1-1) ...
Setting up cuda-cupti-11-1 (11.1.105-1) ...


Setting up cuda-cupti-dev-11-1 (11.1.105-1) ...
Setting up cuda-gdb-11-1 (11.1.105-1) ...
Setting up cuda-command-line-tools-11-1 (11.1.1-1) ...


Processing triggers for libc-bin (2.27-3ubuntu1.4) ...


/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libcuda.so.1 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.1 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libcuda.so is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-ptxjitcompiler.so.1 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-cfg.so.450.51.06 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.450.51.06 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-opencl.so.1 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-allocator.so.450.51.06 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-ptxjitcompiler.so.450.51.06 is empty, not checked.
/sbin/ldconfig.real: File /usr/lib/x86_64-linux-gnu/libnvidia-allocator.so.1 is empty, not checked.
/sbin/l

In [ ]:
%%capture
!conda install cudatoolkit=11.1 pytorch -y

In [ ]:
%cd /tmp

In [ ]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

In [ ]:
!git clone https://github.com/kpu/kenlm

In [ ]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [ ]:

%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [ ]:
%%capture
%cd /tmp/kenlm
!python setup.py install
%cd /tmp

In [ ]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [ ]:
%cd /tmp/fairseq/

In [ ]:
%%capture
!python setup.py install

In [ ]:
%cd /tmp/fairseq/

In [ ]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [ ]:
%%capture
!pip install editdistance

## GAN

In [ ]:
import torch
torch.version.cuda

In [ ]:
torch.backends.cudnn.version()

In [ ]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX CUDA_LAUNCH_BLOCKING=1 fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.keep_last_epochs=20 \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

In [ ]:
!bash rungan.sh